In [1]:
import pandas as pd
from utils.feature_generation import create_data_with_features

In [2]:
path_to_files = "./data/redir"
data = create_data_with_features(path_to_files)

In [3]:
data.head(3)

,is_redirect,len_of_query,num_of_word_in_query,len_of_category,num_of_word_in_category,how_match_brands_name_in_query,how_match_products_name_in_query,mean_word_len_in_category,mean_word_len_in_query,max_word_len_in_category,max_word_len_in_query,min_word_len_in_category,min_word_len_in_query,is_query_long
0,1,14,2,10,1,1,1,10.0,6.5,10,9,10,4,0
1,1,13,2,10,1,1,1,10.0,6.0,10,8,10,4,0
2,1,31,4,10,1,1,3,10.0,7.0,10,9,10,4,0


In [4]:
data.shape

(3003, 14)

In [5]:
X = data.drop(columns='is_redirect')
y = data['is_redirect']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    stratify=y,
    random_state=0,
    shuffle=True,
    test_size=0.2
)

In [8]:
from lazypredict.Supervised import LazyClassifier
lazy_pred_clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
train, test = lazy_pred_clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 30/30 [00:03<00:00,  8.28it/s]


In [9]:
train

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.91,0.91,0.91,0.91,0.15
XGBClassifier,0.91,0.91,0.91,0.91,0.20
RandomForestClassifier,0.90,0.90,0.90,0.90,0.33
ExtraTreesClassifier,0.90,0.90,0.90,0.90,0.26
LabelSpreading,0.87,0.87,0.87,0.87,0.40
LabelPropagation,0.87,0.87,0.87,0.87,0.30
BaggingClassifier,0.87,0.87,0.87,0.87,0.09
DecisionTreeClassifier,0.86,0.86,0.86,0.86,0.03
KNeighborsClassifier,0.86,0.86,0.86,0.86,0.05


In [10]:
test

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.91,0.91,0.91,0.91,0.15
XGBClassifier,0.91,0.91,0.91,0.91,0.20
RandomForestClassifier,0.90,0.90,0.90,0.90,0.33
ExtraTreesClassifier,0.90,0.90,0.90,0.90,0.26
LabelSpreading,0.87,0.87,0.87,0.87,0.40
LabelPropagation,0.87,0.87,0.87,0.87,0.30
BaggingClassifier,0.87,0.87,0.87,0.87,0.09
DecisionTreeClassifier,0.86,0.86,0.86,0.86,0.03
KNeighborsClassifier,0.86,0.86,0.86,0.86,0.05


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline


def make_features_pipeline(n_jobs):
    pipeline = Pipeline([
        ('clf', RandomForestClassifier(n_jobs=n_jobs, random_state=1))
    ])
    return pipeline

In [8]:
from pipeline.training import SCORING, STRONG_CV
from pipeline.training import fit_pipeline, cross_validate_pipeline


N_JOBS = 1
N_JOBS_CV = 4

pipeline = make_features_pipeline(N_JOBS)

result = cross_validate_pipeline(
    pipeline,
    X_train,
    y_train,
    cv=STRONG_CV,
    scoring=SCORING,
    n_jobs=N_JOBS_CV,
    verbose=1,
)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   21.9s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:   24.6s finished


In [13]:
result

{'fit_time': array([0.46843266, 0.41160226, 0.43942857, 0.43771935, 0.39764977,
        0.47907543, 0.41467071, 0.49060035, 0.45106721, 0.42961979,
        0.38684154, 0.41538835, 0.39184022, 0.43915033, 0.36109424,
        0.41566062, 0.43677878, 0.56345391, 0.45416451, 0.43852639,
        0.36994076, 0.40055537, 0.44143057, 0.35478139, 0.36469769,
        0.40836167, 0.36525083, 0.35255814, 0.50183272, 0.46806788,
        0.45787954, 0.38864112, 0.61061168, 0.35428238, 0.43100953,
        0.38482714, 0.39169908, 0.55562305, 0.37521791, 0.38313842,
        0.4589045 , 0.37024331, 0.403404  , 0.4267056 , 0.43066931,
        0.37108684, 0.49328685, 0.49570751, 0.31514072, 0.31854081]),
 'score_time': array([0.05781794, 0.06178832, 0.05764699, 0.05975723, 0.08331418,
        0.07159257, 0.06206059, 0.05819273, 0.05758309, 0.05774879,
        0.0640192 , 0.06765175, 0.05744886, 0.08417654, 0.05685019,
        0.05142045, 0.05651641, 0.07727194, 0.07034683, 0.06260872,
        0.05214477, 

In [9]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('clf', RandomForestClassifier(n_jobs=1, random_state=1))])

In [10]:
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score


print("RF from pipeline:")

print("\nMetrics for train:")
for metric in [roc_auc_score, f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_train, pipeline.predict(X_train)), 4))

print("\nMetrics for test:")
for metric in [roc_auc_score, f1_score, accuracy_score]:
    print(metric.__name__, round(metric(y_test, pipeline.predict(X_test)), 4))

RF from pipeline:

Metrics for train:
roc_auc_score 0.9976
f1_score 0.9973
accuracy_score 0.9975

Metrics for test:
roc_auc_score 0.9
f1_score 0.8929
accuracy_score 0.9018
